In [168]:
import numpy as np 
import pandas as pd
from itertools import permutations
from itertools import combinations
from itertools import combinations_with_replacement

In [169]:
df = pd.read_excel('./CoffeeShopTransactions.xlsx',usecols = ['Item 1','Item 2','Item 3'])
df.head()

,Item 1,Item 2,Item 3
0,Hot chocolate,CaramelBites,Mineral water
1,Cookies,CaramelBites,CaramelBites
2,Coffee,Chocolates,Brownie
3,Coffee,Brownie,Tea
4,Tea,Cookies,Cake


In [170]:
minsup = float(input("Enter Minimum Support :"))
minconf = float(input("Enter Minimum Confidence :"))

Enter Minimum Support :0
Enter Minimum Confidence :0.3


In [171]:
supcount = 0
if(minsup<=1):
    supcount = minsup*(df.size/len(df.iloc[0])) ###### df.size/3
else :
    supcount = (minsup*(df.size/len(df.iloc[0])))/100 ##### df.size)/3
    
if (minconf>1) :
    minconf /=100

In [172]:
print(supcount)

0.0


In [173]:
#get all unique values in dataset
items = pd.unique(df[df.columns].values.ravel()).tolist()
items

['Hot chocolate',
 'CaramelBites',
 'Mineral water',
 'Cookies',
 'Coffee',
 'Chocolates',
 'Brownie',
 'Tea',
 'Cake',
 'Juice']

In [174]:
item_tuples = [(i,) for i in items]
item_tuples

[('Hot chocolate',),
 ('CaramelBites',),
 ('Mineral water',),
 ('Cookies',),
 ('Coffee',),
 ('Chocolates',),
 ('Brownie',),
 ('Tea',),
 ('Cake',),
 ('Juice',)]

In [175]:
# all possible combinations
sets=[]
sets=list(item_tuples)
sets+=list(combinations(items, 2))
sets+= list(combinations(items, 3))
#sets

In [176]:
#all rows of dataframe converted to sets 
df.iloc[:].values
rows = [set(v) for v in df.iloc[:].values]

In [177]:
#sets

In [178]:
#calculate support count for each set
def supCount (freq,sets,rows,supcount):
    removed=[]
    for i in sets:  
        if i in removed : continue
        for j in rows:
            if(set(i).issubset(j)): 
                freq[i]+=1
        #if set is not frequent, put all its super sets in removed list
        if(freq[i]<supcount):
            for k in sets:
                if set(i).issubset(set(k)) : 
                    removed.append(k)
    

In [179]:
# initialize frequency for all sets by 0
freq={i:0 for i in sets}
supCount(freq,sets,rows,supcount)

In [180]:
def isFrequent (items,supcount):
    freqSet = {}
    for key in items :
        if items[key] >= supcount :
            freqSet[key] = items[key]
    return (freqSet)


In [181]:
#get all frequent set where frequency > support count
freqSet = isFrequent(freq,supcount)

In [182]:
print('\n Frequent item sets :\n',len(freqSet))
for key in freqSet :
    print (key,'  ',freqSet[key])


 Frequent item sets :
 175
('Hot chocolate',)    2416
('CaramelBites',)    1893
('Mineral water',)    2075
('Cookies',)    2363
('Coffee',)    6327
('Chocolates',)    1847
('Brownie',)    2214
('Tea',)    3048
('Cake',)    2670
('Juice',)    2197
('Hot chocolate', 'CaramelBites')    293
('Hot chocolate', 'Mineral water')    325
('Hot chocolate', 'Cookies')    373
('Hot chocolate', 'Coffee')    1285
('Hot chocolate', 'Chocolates')    308
('Hot chocolate', 'Brownie')    371
('Hot chocolate', 'Tea')    543
('Hot chocolate', 'Cake')    474
('Hot chocolate', 'Juice')    362
('CaramelBites', 'Mineral water')    217
('CaramelBites', 'Cookies')    312
('CaramelBites', 'Coffee')    1126
('CaramelBites', 'Chocolates')    191
('CaramelBites', 'Brownie')    280
('CaramelBites', 'Tea')    471
('CaramelBites', 'Cake')    359
('CaramelBites', 'Juice')    233
('Mineral water', 'Cookies')    347
('Mineral water', 'Coffee')    1209
('Mineral water', 'Chocolates')    235
('Mineral water', 'Brownie')    

In [185]:
#calculate confidence of every set 
def conf (freqSet,minconf):
    for key in freqSet :
        #can't calculate confidence of frequent set of one element
        if len(key)==1 : continue
        elif len(key)==2 :
            #confidence of frequent sets of 2 elements, print strong only if confidence (set) > min confidence 
            if(freqSet[key]/freqSet[(key[0],)]>minconf) : 
                print( key[0] ,'-->' , key[1], '=', freqSet[key]/freqSet[(key[0],)], '   (Strong)')
            else : print( key[0] ,'-->' , key[1], '=', freqSet[key]/freqSet[(key[0],)])
            if(freqSet[key]/freqSet[(key[1],)]>minconf) : 
                print( key[1] ,'-->' , key[0], '=', freqSet[key]/freqSet[(key[1],)], '   (Strong)')
            else : print( key[1] ,'-->' , key[0], '=', freqSet[key]/freqSet[(key[1],)])
            
        elif len(key)==3 :
            #confidence of frequent sets of 3 elements, print strong only if confidence (set) > min confidence 
            temp = [j for j in permutations(key,len(key)-1)]
            for x in range (len(temp)):
                for m in range (len(key)):
                    if key[m] not in temp[x] and (temp[x] in freqSet) :
                        if freqSet[key]/freqSet[(key[m],)]>=minconf:
                            print( key[m] ,'-->' , temp[x] , '=', freqSet[key]/freqSet[(key[m],)], '   (Strong)')
                        else :
                            print( key[m] ,'>' , temp[x] , '=', freqSet[key]/freqSet[(key[m],)])
                        if freqSet[key]/freqSet[temp[x]]>=minconf:
                            print( temp[x] ,'>' , key[m] , '=', freqSet[key]/freqSet[temp[x]], '   (Strong)')
                        else :
                             print( temp[x] ,'>' , key[m] , '=', freqSet[key]/freqSet[temp[x]])
            

In [186]:
conf(freqSet,minconf)

Hot chocolate --> CaramelBites = 0.12127483443708609
CaramelBites --> Hot chocolate = 0.15478077126254622
Hot chocolate --> Mineral water = 0.13451986754966888
Mineral water --> Hot chocolate = 0.1566265060240964
Hot chocolate --> Cookies = 0.15438741721854304
Cookies --> Hot chocolate = 0.15785019043588658
Hot chocolate --> Coffee = 0.5318708609271523    (Strong)
Coffee --> Hot chocolate = 0.20309783467678205
Hot chocolate --> Chocolates = 0.12748344370860928
Chocolates --> Hot chocolate = 0.16675690308608554
Hot chocolate --> Brownie = 0.15355960264900662
Brownie --> Hot chocolate = 0.16757000903342367
Hot chocolate --> Tea = 0.22475165562913907
Tea --> Hot chocolate = 0.1781496062992126
Hot chocolate --> Cake = 0.19619205298013245
Cake --> Hot chocolate = 0.17752808988764046
Hot chocolate --> Juice = 0.1498344370860927
Juice --> Hot chocolate = 0.16477014110150204
CaramelBites --> Mineral water = 0.11463285789751716
Mineral water --> CaramelBites = 0.10457831325301205
CaramelBites -

('CaramelBites', 'Cake') > Brownie = 0.07242339832869081
CaramelBites > ('Brownie', 'Cake') = 0.013734812466983624
('Brownie', 'Cake') > CaramelBites = 0.057777777777777775
Juice > ('CaramelBites', 'Brownie') = 0.006827492034592627
('CaramelBites', 'Brownie') > Juice = 0.05357142857142857
Brownie > ('CaramelBites', 'Juice') = 0.006775067750677507
('CaramelBites', 'Juice') > Brownie = 0.06437768240343347
CaramelBites > ('Brownie', 'Juice') = 0.00792393026941363
('Brownie', 'Juice') > CaramelBites = 0.04398826979472141
Cake > ('CaramelBites', 'Tea') = 0.015355805243445693
('CaramelBites', 'Tea') > Cake = 0.0870488322717622
Tea > ('CaramelBites', 'Cake') = 0.013451443569553806
('CaramelBites', 'Cake') > Tea = 0.11420612813370473
CaramelBites > ('Tea', 'Cake') = 0.021658742736397254
('Tea', 'Cake') > CaramelBites = 0.06856187290969899
Juice > ('CaramelBites', 'Tea') = 0.015930814747382796
('CaramelBites', 'Tea') > Juice = 0.07430997876857749
Tea > ('CaramelBites', 'Juice') = 0.011482939632